In [1]:
import pandas as pd
import numpy as np
import random as rd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
import math

In [2]:
df = pd.read_csv("../input_data/intrusion_detection/data.csv")
Y = df.xAttack
X = df.drop(['xAttack'],axis=1)
X = (X - X.mean())/X.std()

In [3]:
cov_x = np.cov(X.T)

U,S,V = np.linalg.svd(cov_x)
S_sum = float(np.sum(S))

running_sum = 0
num_of_comp = 0
for i in xrange(len(S)):
    running_sum += S[i]
    if running_sum  / S_sum  >= 0.90:
        num_of_comp = i+1
        break

U_red = U[:,:num_of_comp]

Z = np.matmul(U_red.T, X.T)
Z = Z.T
Z_new = pd.DataFrame( Z,columns=[ "pc"+str(i) for i in xrange(Z.shape[1]) ] )

In [4]:
gmm = GaussianMixture(n_components=5, n_init=10 )

In [5]:
gmm.fit(Z_new)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=5, n_init=10, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [6]:
# print gmm.means_
# print gmm.covariances_
class_var =  gmm.predict(Z_new)
print class_var.shape

(24998,)


In [7]:
def purity(df,K):
    pur_dict = {}
    for i in xrange(K):
        sub_tab = df[ df['pred_Y'] == i ]
        name,count = np.unique(sub_tab['xAttack'],return_counts=True)
        mx_ind = np.argmax(count)
        print i , name[mx_ind]
        pur_dict[i] = count[mx_ind] / float(len(sub_tab))
        print name
        print count
        
    return pur_dict

In [8]:
Z_new = pd.DataFrame( Z,columns=[ "pc"+str(i) for i in xrange(Z.shape[1]) ] )
Z_new = pd.concat([Z_new, Y], axis=1)

pred_Y = pd.DataFrame( class_var,columns=[ 'pred_Y' ] )
Z_final = pd.concat([Z_new, pred_Y],axis=1)

In [9]:
purity_dict = purity(Z_final,5)
print purity_dict

0 normal
['dos' 'normal' 'probe' 'r2l']
[ 771 2166 1154   91]
1 dos
['dos' 'normal' 'probe']
[1364  550 1034]
2 dos
['dos']
[6136]
3 normal
['dos' 'normal' 'probe' 'r2l' 'u2r']
[ 268 1043   40   91    7]
4 normal
['dos' 'normal' 'probe' 'r2l' 'u2r']
[ 575 9605   85   15    3]
{0: 0.5179340028694405, 1: 0.4626865671641791, 2: 1.0, 3: 0.7198067632850241, 4: 0.9340659340659341}
